In [63]:
import requests
import zipfile
import os
import sys
import io
import pandas as pd
from io import BytesIO
from tqdm import tqdm
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
import logging
import time
import geopandas as gpd
import folium

In [32]:
main_path=os.path.dirname(os.path.abspath(os.getcwd()))
data_path = os.path.join(main_path, 'data', 'interim')
censo_path=os.path.join(data_path,'censo')
shp_path=os.path.join(data_path,'shp')

In [33]:
censo_ent_path=os.path.join(censo_path,'censo_ent_tidy_data.csv')
censo_mun_path=os.path.join(censo_path,'censo_mun_tidy_data.csv')

In [34]:
shp_ent_path=os.path.join(shp_path,'shp_ent_tidy_data.csv')
shp_mun_path=os.path.join(shp_path,'shp_mun_tidy_data.csv')

In [35]:
df_cve_ent = pd.read_csv(censo_ent_path, encoding='utf-8')
df_cve_mun = pd.read_csv(censo_mun_path, encoding='utf-8')

In [36]:
df_cve_ent.head()

,cvegeo,pob_tot,rel_h_m
0,1,1425607,95.58
1,2,3769020,101.72
2,3,798447,103.39
3,4,928363,96.93
4,5,3146771,98.77


In [37]:
df_cve_mun.head()

,cvegeo,pob_tot,rel_h_m
0,1001,948990,94.90
1,1002,51536,96.14
2,1003,58250,96.21
3,1004,17000,95.22
4,1005,129929,97.73


In [38]:
gdf_ent = gpd.read_file(shp_ent_path) 
gdf_mun = gpd.read_file(shp_mun_path) 

In [39]:
gdf_ent['cvegeo']=gdf_ent['cvegeo'].astype(int)
gdf_mun['cvegeo']=gdf_mun['cvegeo'].astype(int)

In [51]:
gdf_ent = gdf_ent.rename(columns={'geo': 'geometry'})
gdf_mun = gdf_mun.rename(columns={'geo': 'geometry'})

In [52]:
df_cve_mun.head()

,cvegeo,pob_tot,rel_h_m
0,1001,948990,94.90
1,1002,51536,96.14
2,1003,58250,96.21
3,1004,17000,95.22
4,1005,129929,97.73


In [53]:
gdf_mun.head()

,cvegeo,nom_geo,geometry
0,1001,Aguascalientes,POLYGON ((-102.10641224967047 22.0603544202862...
1,1002,Asientos,POLYGON ((-102.05189344976499 22.2914352898702...
2,1003,Calvillo,POLYGON ((-102.68568845017741 22.0996273097580...
3,1004,Cosío,POLYGON ((-102.28786517984673 22.4164900503151...
4,1005,Jesús María,POLYGON ((-102.33567757017497 22.0506652195055...


In [54]:
gdf_mun_pob = gdf_mun.merge(df_cve_mun, on=['cvegeo'], how='left')
gdf_mun_pob=gdf_mun_pob[['cvegeo','nom_geo','pob_tot','rel_h_m','geometry']] # desired order
gdf_mun_pob.head()

,cvegeo,nom_geo,pob_tot,rel_h_m,geometry
0,1001,Aguascalientes,948990,94.90,POLYGON ((-102.10641224967047 22.0603544202862...
1,1002,Asientos,51536,96.14,POLYGON ((-102.05189344976499 22.2914352898702...
2,1003,Calvillo,58250,96.21,POLYGON ((-102.68568845017741 22.0996273097580...
3,1004,Cosío,17000,95.22,POLYGON ((-102.28786517984673 22.4164900503151...
4,1005,Jesús María,129929,97.73,POLYGON ((-102.33567757017497 22.0506652195055...


In [58]:
gdf_ent_pob = gdf_ent.merge(df_cve_ent, on=['cvegeo'], how='left')
gdf_ent_pob=gdf_ent_pob[['cvegeo','nom_geo','pob_tot','rel_h_m','geometry']] # desired order
gdf_ent_pob.head()

,cvegeo,nom_geo,pob_tot,rel_h_m,geometry
0,1,Aguascalientes,1425607,95.58,POLYGON ((-102.28786517984673 22.4164900503151...
1,2,Baja California,3769020,101.72,MULTIPOLYGON (((-112.3006708050326 28.38702499...
2,3,Baja California Sur,798447,103.39,MULTIPOLYGON (((-109.8945056436797 22.87475837...
3,4,Campeche,928363,96.93,MULTIPOLYGON (((-91.95649652028466 20.19699502...
4,5,Coahuila de Zaragoza,3146771,98.77,POLYGON ((-102.31079265000164 29.8769485699491...


In [64]:
# Define the style function to color the polygons based on rel_h_m values
def style_function(feature):
    # Convert rel_h_m to numeric, coerce errors to NaN
    rel_h_m_value = pd.to_numeric(feature['properties']['rel_h_m'], errors='coerce')

    if pd.notna(rel_h_m_value):  # Check if the value is not NaN
        if 99 <= rel_h_m_value <= 101:
            fill_color = 'green'
        elif rel_h_m_value > 101:
            fill_color = 'blue'
        else:
            fill_color = 'pink'
    else:
        fill_color = 'gray'  # Default color for NaN values

    return {
        'fillColor': fill_color,
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.5,
    }

In [ ]:
# Create a folium map
m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)  # Center the map on Mexico

# Add polygons to the map
folium.GeoJson(
     gdf_ent_pob,
     style_function=style_function,
     tooltip=folium.GeoJsonTooltip(
        fields=['nom_geo', 'pob_tot', 'rel_h_m'],  
        aliases=['Entity:','Population:','Male per 100 Female:'], # Indicates the number of men per 100 women.
        localize=True
    )
).add_to(m)

# Display the map
m.save('interactive_map.html')
m

In [ ]:
# Create a folium map
m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)  # Center the map on Mexico, adjust location as needed

# Add polygons to the map
folium.GeoJson(
    gdf_mun_pob,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['nom_geo','pob_tot','rel_h_m'], 
        aliases=['Municipality:','Population:','Male per 100 Female:'], # Indicates the number of men per 100 women.
        localize=True
    )
).add_to(m)

# Display the map
m.save('interactive_map.html')
m